In [1]:
#importing libraries
import pandas as pd
from tensorflow.keras.optimizers import RMSprop,Adam,Adagrad,SGD
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras import Model
from os import getcwd
from keras.layers import Dense, Activation, BatchNormalization
import tensorflow as tf

##### Kaggle dataset import

In [2]:
!pip install ipython-autotime

In [3]:
 pip install -q kaggle

In [13]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kushagra180","key":"d26c61c002f54027c1228ad3ba86bcb4"}'}

In [14]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [15]:
! cp kaggle.json ~/.kaggle/

In [16]:
! chmod 600 ~/.kaggle/kaggle.json

In [17]:
! kaggle datasets download -d kmader/parkinsons-drawings

 84% 35.0M/41.4M [00:00<00:00, 27.6MB/s]
100% 41.4M/41.4M [00:00<00:00, 60.8MB/s]


In [18]:
!unzip "/content/parkinsons-drawings.zip"

Archive:  /content/parkinsons-drawings.zip
  inflating: drawings/spiral/testing/healthy/V01HE01.png  
  inflating: drawings/spiral/testing/healthy/V02HE01.png  
  inflating: drawings/spiral/testing/healthy/V03HE1.png  
  inflating: drawings/spiral/testing/healthy/V04HE01.png  
  inflating: drawings/spiral/testing/healthy/V05HE01.png  
  inflating: drawings/spiral/testing/healthy/V06HE01.png  
  inflating: drawings/spiral/testing/healthy/V07HE01.png  
  inflating: drawings/spiral/testing/healthy/V08HE01.png  
  inflating: drawings/spiral/testing/healthy/V09HE01.png  
  inflating: drawings/spiral/testing/healthy/V10HE01.png  
  inflating: drawings/spiral/testing/healthy/V11HE01.png  
  inflating: drawings/spiral/testing/healthy/V55HE12.png  
  inflating: drawings/spiral/testing/healthy/V55HE13.png  
  inflating: drawings/spiral/testing/healthy/V55HE14.png  
  inflating: drawings/spiral/testing/healthy/V55HE15.png  
  inflating: drawings/spiral/testing/parkinson/V01PE01.png  
  inflating:

In [20]:
TRAINING_DIR = '/content/spiral/training'
TESTING_DIR = '/content/spiral/testing'

In [40]:
TARGET_SIZE = 128
BATCH_SIZE = 64

In [41]:

train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range = 40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    validation_split=0.1,
    zoom_range=.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size = BATCH_SIZE,
    class_mode='binary',
    target_size = (TARGET_SIZE,TARGET_SIZE)
)


validation_generator = test_datagen.flow_from_directory(
    TESTING_DIR,
    batch_size = BATCH_SIZE,
    class_mode='binary',
    target_size=(TARGET_SIZE,TARGET_SIZE)
)   

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [80]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor = 'val_accuracy', patience = 20, verbose=0, mode='max')
mcp_save_xc = ModelCheckpoint(filepath = 'mdl_wts.hdf5', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
mcp_save_ens = ModelCheckpoint(filepath = 'mdl_wts_en.h5', save_best_only = True, monitor = 'val_accuracy', mode = 'max')
mcp_save_ens1 = ModelCheckpoint(filepath =' mdl_wts_en1.h5', save_best_only = True, monitor = 'val_loss', mode = 'min')
reduce_lr_loss = ReduceLROnPlateau(monitor = 'val_loss', patience=2, verbose=1, min_delta = 1e-4, mode = 'min')

time: 4.36 ms (started: 2021-07-29 16:27:55 +00:00)


In [61]:
from keras.applications.densenet import DenseNet201

In [67]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(TARGET_SIZE,TARGET_SIZE,3),include_top=False,weights='imagenet',pooling='avg')

In [68]:
pretrained_model3.trainable = False

In [81]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(1, activation='sigmoid')(x3)
model3 = tf.keras.Model(inputs=inputs3, outputs=outputs3)

time: 56.1 ms (started: 2021-07-29 16:28:01 +00:00)


In [82]:
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

time: 17 ms (started: 2021-07-29 16:28:02 +00:00)


In [83]:
history_own = model3.fit(
      train_generator,  
      callbacks = [mcp_save_xc, reduce_lr_loss,earlyStopping],
      validation_data = validation_generator,
      epochs = 50,
      verbose = 1,
 )

%load_ext autotime

Epoch 1/50
2/2 [==============================] - 12s 3s/step - loss: 0.6281 - accuracy: 0.6389 - val_loss: 0.7796 - val_accuracy: 0.7000
Epoch 2/50
2/2 [==============================] - 1s 586ms/step - loss: 0.5275 - accuracy: 0.8194 - val_loss: 0.5932 - val_accuracy: 0.6000
Epoch 3/50
2/2 [==============================] - 1s 202ms/step - loss: 0.6200 - accuracy: 0.6250 - val_loss: 0.3966 - val_accuracy: 0.8667
Epoch 4/50
2/2 [==============================] - 1s 195ms/step - loss: 0.4927 - accuracy: 0.7778 - val_loss: 0.5291 - val_accuracy: 0.8000
Epoch 5/50
2/2 [==============================] - 1s 564ms/step - loss: 0.5854 - accuracy: 0.7361 - val_loss: 0.5362 - val_accuracy: 0.8000

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 6/50
2/2 [==============================] - 1s 180ms/step - loss: 0.5580 - accuracy: 0.7222 - val_loss: 0.5018 - val_accuracy: 0.8000
Epoch 7/50
2/2 [==============================] - 1s 590ms/step - loss: 0.5025 -